# Lab 2 Extension: KV Cache Offloading with KVBM

## Overview

In this extension to Lab 2, you will:
- Learn about Dynamo's KV Block Manager (KVBM) for multi-tier memory management
- Deploy baseline and KVBM-enabled serving configurations
- Run benchmarks demonstrating prefix cache reuse improvements
- Compare performance with and without KV cache offloading

**Prerequisites**: Complete Lab 1 and understand basic Dynamo deployment

## Duration: ~60 minutes

---

## Section 1: Understanding KVBM

### What Is the KV Block Manager?

The **KV Block Manager (KVBM)** is Dynamo's sophisticated memory management system that extends vLLM's capacity beyond GPU memory limitations by intelligently managing KV cache blocks across multiple storage tiers.

### The Problem: GPU Memory Constraints

Traditional LLM serving faces hard limits:

```text
Without KVBM:
  Request 1 (20K tokens) → GPU KV Cache [████████] (fills GPU)
  Request 2 (20K tokens) → GPU KV Cache [████████] (fills more)
  Request 3 (20K tokens) → GPU KV Cache [████████] (GPU full!)
  Request 4 (20K tokens) → Must evict Request 1's cache ❌ (lost forever)
  Request 5 reuses Request 1's prefix → Must recompute all 20K tokens 🐌 (slow!)
```

### The Solution: Multi-Tier Memory Management

KVBM solves this by offloading inactive KV blocks to CPU memory (and disk):

```text
With KVBM + 20GB CPU Cache:
  Request 1 (20K tokens) → GPU KV Cache [████████]
  Request 2 (20K tokens) → GPU KV Cache [████████]
  Request 3 (20K tokens) → GPU KV Cache [████████] (GPU full!)
  Request 4 (20K tokens) → Offload Request 1 to CPU, add Request 4 to GPU ✅
  Request 5 reuses Request 1's prefix → Fetch from CPU to GPU ⚡ (fast!)
```

### Key Benefits

| Aspect | Without KVBM | With KVBM |
|--------|-------------|-----------|
| **Capacity** | Limited to GPU memory (~80GB) | GPU + CPU + Disk (hundreds of GB) |
| **Prefix Reuse** | Lost when evicted from GPU | Preserved in CPU/disk for reuse |
| **Request Handling** | Reject when GPU full | Offload to slower tiers gracefully |
| **Long Context** | Competes for GPU memory | Can overflow to CPU/disk |

### Architecture

```text
┌─────────────────────────────────────┐
│  vLLM Engine with KVBM Connector    │
└─────────────────┬───────────────────┘
                  │
      ┌───────────▼──────────┐
      │   KV Block Manager   │
      └───────────┬──────────┘
                  │
      ┌───────────┼──────────┐
      │           │          │
  ┌───▼────┐  ┌──▼───┐  ┌──▼────┐
  │  GPU   │  │ CPU  │  │ Disk  │
  │ (Fast) │  │(Mid) │  │(Slow) │
  │ ~80GB  │  │20GB+ │  │100GB+ │
  └────────┘  └──────┘  └───────┘
```

---

## Section 2: Deploy Baseline Configuration (No KVBM)

### Objectives
- Deploy vLLM with standard prefix caching (no KVBM) on Kubernetes
- Run benchmark demonstrating prefix cache eviction behavior
- Establish baseline performance metrics

**Note:** We'll use constrained GPU memory (20% utilization) to force prefix eviction, demonstrating the problem KVBM solves.

### Step 1: Review Baseline Deployment YAML

The baseline deployment. Review `kvbm_baseline.yaml`:

Key configuration:
- **Connector**: NIXL (standard vLLM prefix caching)
- **GPU utilization**: 20% (constrains memory to force eviction)
- **Model**: Qwen/Qwen3-4B
- **Prefix caching**: Enabled (but only in GPU memory)

### Step 2: Deploy Baseline Configuration

In [4]:
%%bash
# Deploy baseline (no KVBM)
kubectl apply -f kvbm_baseline.yaml --namespace $NAMESPACE

echo "✓ Baseline deployment created"
echo "Models are pre-cached, this will take 1-2 minutes for model loading..."
echo ""
echo "Configuration:"
echo "  - Connector: NIXL (standard vLLM)"
echo "  - GPU utilization: 20% (forces prefix eviction)"
echo "  - Prefix caching: Enabled (but only in GPU memory)"
echo "  - Model: /models/Qwen3-4B (pre-cached)"

dynamographdeployment.nvidia.com/vllm-kvbm-baseline configured
✓ Baseline deployment created
Models are pre-cached, this will take 1-2 minutes for model loading...

Configuration:
  - Connector: NIXL (standard vLLM)
  - GPU utilization: 20% (forces prefix eviction)
  - Prefix caching: Enabled (but only in GPU memory)
  - Model: /models/Qwen3-4B (pre-cached)


### Step 3: Monitor Baseline Deployment

Wait for the pods to be ready:

In [6]:
%%bash
# Check deployment status
echo "Baseline deployment:"
kubectl get pods -n $NAMESPACE -l nvidia.com/dynamo-namespace=vllm-kvbm-baseline

echo ""
echo "Wait for all pods to show 1/1 Ready"

Baseline deployment:
NAME                                                   READY   STATUS    RESTARTS   AGE
vllm-kvbm-baseline-frontend-7d6bdf4d6d-7q22h           1/1     Running   0          4m19s
vllm-kvbm-baseline-vllmdecodeworker-7fdd5c9688-wf7q2   0/1     Pending   0          4m19s

Wait for all pods to show 1/1 Ready


### Step 4: Verify Baseline Deployment

Ensure the worker is healthy:

In [ ]:
%%bash
# Count ready workers
BASELINE_READY=$(kubectl get pods -n $NAMESPACE -l nvidia.com/dynamo-namespace=vllm-kvbm-baseline,nvidia.com/dynamo-component-type=worker --field-selector=status.phase=Running -o json | jq '[.items[] | select(.status.conditions[] | select(.type=="Ready" and .status=="True"))] | length')

echo "Baseline workers ready: $BASELINE_READY/1"

if [ "$BASELINE_READY" = "1" ]; then
    echo "✓ Baseline deployment is healthy - ready to benchmark"
else
    echo "⚠️  Worker not ready. Wait before benchmarking."
fi

---

## Section 3: Benchmark Baseline Performance

### Understanding the Benchmark

We'll use `aiperf` with these key parameters:
- **`--prefix-prompt-length 20000`**: Each request has a 20,000 token prefix
- **`--prompt-prefix-pool-size 5`**: Five different prefixes (cycles through them)
- **`--request-count 30`**: 30 total requests (each prefix reused multiple times)

This creates memory pressure by cycling through large prefixes, forcing eviction.

### Step 1: Set Up Port Forwarding

In [ ]:
%%bash --bg
# Forward baseline frontend
kubectl port-forward deployment/vllm-kvbm-baseline-frontend $USER_FRONTEND_PORT:8000 -n $NAMESPACE &

echo "✓ Baseline frontend: localhost:${USER_FRONTEND_PORT}"
sleep 5

### Step 2: Install AIPerf

In [ ]:
%%bash
# Install AIPerf if not already available
pip install aiperf -q

echo "✓ AIPerf installed"

### Step 3: Run Baseline Benchmark

In [ ]:
%%bash
# Run baseline benchmark with prefix cycling
aiperf profile \
  -m "Qwen/Qwen3-4B" \
  --endpoint-type chat \
  --streaming \
  -u http://localhost:${USER_FRONTEND_PORT} \
  --synthetic-input-tokens-mean 128 \
  --synthetic-input-tokens-stddev 0 \
  --prefix-prompt-length 20000 \
  --prompt-prefix-pool-size 5 \
  --output-tokens-mean 50 \
  --concurrency 1 \
  --request-count 30 \
  --tokenizer "Qwen/Qwen3-4B" \
  --artifact-dir ./results_baseline_kvbm

echo ""
echo "✓ Baseline benchmark complete"
echo "Results saved to: ./results_baseline_kvbm/"

### Step 4: Save Baseline Metrics

Take note of key metrics from the output:
- **Time to First Token (TTFT)**: Average and p99
- **TTFT Variance**: High variance indicates prefix cache eviction/recomputation
- **Request Latency**: Total end-to-end time
- **Throughput**: Requests and tokens per second

### Step 5: Delete Baseline Deployment

Clean up before starting the KVBM experiment:

In [ ]:
%%bash
# Stop port forward
pkill -f "port-forward.*vllm-kvbm-baseline"

# Delete deployment
kubectl delete dynamographdeployment vllm-kvbm-baseline -n $NAMESPACE

echo "✓ Baseline deployment deleted"
echo "Waiting for pods to terminate..."
sleep 10

# Verify cleanup
kubectl get pods -n $NAMESPACE -l nvidia.com/dynamo-namespace=vllm-kvbm-baseline

---

## Section 4: Deploy KVBM Configuration

### Objectives
- Configure KVBM with CPU cache offloading
- Deploy vLLM with KVBM connector on Kubernetes
- Understand KVBM configuration parameters

### Step 1: Review KVBM Deployment YAML

The KVBM deployment uses the KVBM connector with CPU cache offloading. Review `kvbm_enabled.yaml`:

Key configuration:
- **Connector**: KVBM (with CPU offloading)
- **GPU utilization**: 20% (same constraint as baseline)
- **CPU cache**: 20GB (`DYN_KVBM_CPU_CACHE_GB=20`)
- **Model**: Qwen/Qwen3-4B (same as baseline)
- **Prefix caching**: Enabled (GPU + CPU tiers)

### Step 2: Deploy KVBM Configuration

**Note:** Models are pre-cached in the shared storage, so this should start quickly (1-2 minutes).

In [ ]:
%%bash
# Deploy KVBM-enabled configuration
kubectl apply -f kvbm_enabled.yaml --namespace $NAMESPACE

echo "✓ KVBM deployment created"
echo "Models are pre-cached, this will take 1-2 minutes for model loading..."
echo ""
echo "Configuration:"
echo "  - Connector: KVBM (with CPU offloading)"
echo "  - GPU utilization: 20% (forces prefix eviction)"
echo "  - CPU cache: 20GB (preserves evicted prefixes)"
echo "  - Prefix caching: Enabled (GPU + CPU tiers)"
echo "  - Model: /models/Qwen3-4B (pre-cached)"

### Step 3: Monitor KVBM Deployment

Wait for the pods to be ready:

In [ ]:
%%bash
# Check deployment status
echo "KVBM deployment:"
kubectl get pods -n $NAMESPACE -l nvidia.com/dynamo-namespace=vllm-kvbm-enabled

echo ""
echo "Wait for all pods to show 1/1 Ready"

### Step 4: Verify KVBM Deployment

Ensure the worker is healthy:

In [ ]:
%%bash
# Count ready workers
KVBM_READY=$(kubectl get pods -n $NAMESPACE -l nvidia.com/dynamo-namespace=vllm-kvbm-enabled,nvidia.com/dynamo-component-type=worker --field-selector=status.phase=Running -o json | jq '[.items[] | select(.status.conditions[] | select(.type=="Ready" and .status=="True"))] | length')

echo "KVBM workers ready: $KVBM_READY/1"

if [ "$KVBM_READY" = "1" ]; then
    echo "✓ KVBM deployment is healthy - ready to benchmark"
else
    echo "⚠️  Worker not ready. Wait before benchmarking."
fi

---

## Section 5: Benchmark KVBM Performance

### Step 1: Set Up Port Forwarding

In [ ]:
%%bash --bg
# Forward KVBM frontend
kubectl port-forward deployment/vllm-kvbm-enabled-frontend $USER_FRONTEND_PORT:8000 -n $NAMESPACE &

echo "✓ KVBM frontend: localhost:${USER_FRONTEND_PORT}"
sleep 5

### Step 2: Run KVBM Benchmark

Run the exact same benchmark with KVBM enabled:

In [ ]:
%%bash
# Run KVBM benchmark with identical parameters
aiperf profile \
  -m "Qwen/Qwen3-4B" \
  --endpoint-type chat \
  --streaming \
  -u http://localhost:${USER_FRONTEND_PORT} \
  --synthetic-input-tokens-mean 128 \
  --synthetic-input-tokens-stddev 0 \
  --prefix-prompt-length 20000 \
  --prompt-prefix-pool-size 5 \
  --output-tokens-mean 50 \
  --concurrency 1 \
  --request-count 30 \
  --tokenizer "Qwen/Qwen3-4B" \
  --artifact-dir ./results_kvbm

echo ""
echo "✓ KVBM benchmark complete"
echo "Results saved to: ./results_kvbm/"

---

## Section 6: Analyze and Compare Results

### Key Metrics to Compare

| Metric | What It Shows | What to Look For |
|--------|---------------|------------------|
| **TTFT Average** | Average time to first token | Lower with KVBM (less recomputation) |
| **TTFT p99** | Worst-case latency | Much lower with KVBM (no full recomputation) |
| **TTFT Variance** | Consistency across requests | Lower with KVBM (stable cache access) |
| **Request Latency** | Total request time | Lower with KVBM |
| **Throughput** | Requests/sec and tokens/sec | Higher with KVBM |

### Understanding the Results

**What Happens Without KVBM (Baseline):**

1. Requests 1-4 arrive with different 20K token prefixes
2. GPU KV cache fills up completely
3. Request 5 arrives → must evict oldest prefix blocks (Request 1)
4. Later request reuses Request 1's prefix → **must recompute all 20K tokens** 🐌
5. High TTFT variance as prefixes are repeatedly evicted and recomputed

**What Happens With KVBM:**

1. Requests 1-4 arrive with different 20K token prefixes
2. GPU KV cache fills up completely
3. Request 5 arrives → **offloads** Request 1's blocks to CPU (preserved) ✅
4. Later request reuses Request 1's prefix → **fetches from CPU to GPU** ⚡
5. Consistent TTFT as prefixes are preserved and quickly restored

### Example Comparison

```text
Baseline (No KVBM):
  TTFT avg: 3,200 ms     p99: 12,500 ms  (huge variance!)
  Request Latency avg: 4,800 ms
  Throughput: 6.2 req/sec

  📊 High p99 indicates frequent full recomputation of 20K token prefixes

KVBM (20GB CPU Cache):
  TTFT avg: 1,800 ms     p99: 3,200 ms   (44% faster avg, 74% faster p99! ✅)
  Request Latency avg: 3,400 ms  (29% faster ✅)
  Throughput: 8.8 req/sec  (42% higher ✅)

  📊 Lower p99 shows prefixes preserved in CPU and quickly restored
```

### Why KVBM Improves Performance

1. **Eliminates Redundant Computation**: Prefixes stored in CPU don't need recomputation
2. **Reduces TTFT Variance**: CPU fetch (~200ms) vs full recompute (~10s)
3. **Increases Effective Capacity**: 20GB CPU cache extends usable KV cache beyond GPU
4. **Enables Better Resource Utilization**: GPU focused on active inference, CPU stores inactive blocks

### When KVBM Provides Maximum Benefit

**Enable KVBM when:**
- Long prefixes (>5000 tokens) are reused across requests
- System prompts or templates are shared across many requests
- Multi-turn conversations with context carryover
- Document Q&A where the same document is queried multiple times
- GPU memory is constrained and cache eviction is frequent

**KVBM provides less benefit when:**
- Every request has a unique prompt with no reuse
- Short prompts (<1000 tokens) where cache overhead dominates
- Abundant GPU memory where eviction rarely occurs
- Workloads are I/O bound rather than compute bound

---

## Section 7: Optional - Enable Disk Offloading

For even larger capacity, enable disk tier offloading by modifying the KVBM deployment:

### Step 1: Create Multi-Tier KVBM Deployment

Modify your `kvbm_enabled.yaml` to add disk tier configuration:

```yaml
      envs:
        - name: DYN_LOG
          value: "info"
        - name: DYN_KVBM_CPU_CACHE_GB
          value: "16"  # 16GB CPU cache
        - name: DYN_KVBM_DISK_CACHE_GB
          value: "100"  # 100GB disk cache
        - name: DYN_KVBM_DISK_CACHE_DIR
          value: "/mnt/nvme/kvbm_cache"  # Use fast SSD
        - name: DYN_KVBM_LEADER_WORKER_INIT_TIMEOUT_SECS
          value: "600"  # Increased timeout for large caches
```

And add a volume mount for disk cache:

```yaml
        volumes:
        - name: local-model-cache
          hostPath:
            path: /data/huggingface-cache
            type: DirectoryOrCreate
        - name: kvbm-disk-cache
          hostPath:
            path: /mnt/nvme/kvbm_cache
            type: DirectoryOrCreate
        mainContainer:
          volumeMounts:
          - name: local-model-cache
            mountPath: /root/.cache
          - name: kvbm-disk-cache
            mountPath: /mnt/nvme/kvbm_cache
```

### Step 2: Deploy Multi-Tier Configuration

In [ ]:
%%bash
# Apply the updated YAML
kubectl apply -f kvbm_enabled.yaml --namespace $NAMESPACE

echo "✓ Multi-tier KVBM configuration deployed"
echo ""
echo "Configuration:"
echo "  CPU cache:  16GB"
echo "  Disk cache: 100GB (on /mnt/nvme/kvbm_cache)"

**Note:** Disk offloading is slower than CPU but provides massive capacity for very large workloads.

---

## Section 8: Monitoring KVBM (Optional)

### View KVBM Logs

Watch KVBM operations in real-time:

In [ ]:
%%bash
# Get KVBM worker pod name
KVBM_POD=$(kubectl get pods -n $NAMESPACE -l nvidia.com/dynamo-namespace=vllm-kvbm-enabled,nvidia.com/dynamo-component-type=worker -o jsonpath='{.items[0].metadata.name}')

echo "Viewing logs from KVBM worker: $KVBM_POD"
echo ""
kubectl logs -n $NAMESPACE $KVBM_POD --tail=50 --follow

Look for messages like:

```text
[INFO] KVBM initialized with 20GB CPU cache
[DEBUG] Offloading block with sequence hash 1234567890 to target pool.
[DEBUG] Onboarding 3 blocks to target pool.
[DEBUG] Adding blocks to pool
[DEBUG] Returning blocks to pool
```

### Monitor Resource Usage

Watch GPU and memory utilization from the worker pod:

In [ ]:
%%bash
# Execute nvidia-smi inside the worker pod
KVBM_POD=$(kubectl get pods -n $NAMESPACE -l nvidia.com/dynamo-namespace=vllm-kvbm-enabled,nvidia.com/dynamo-component-type=worker -o jsonpath='{.items[0].metadata.name}')

kubectl exec -n $NAMESPACE $KVBM_POD -- nvidia-smi

# Check system memory
kubectl exec -n $NAMESPACE $KVBM_POD -- free -h

---

## Section 9: Cleanup

When you're done with the KVBM experiments:

In [ ]:
%%bash
# Delete KVBM deployment (baseline was already deleted)
kubectl delete dynamographdeployment vllm-kvbm-enabled -n $NAMESPACE

# Stop port forwards
pkill -f "port-forward.*vllm-kvbm"

echo "✓ Cleanup complete"
echo "Note: Your namespace and platform from Lab 1 are still running"
echo "KVBM cache is automatically cleaned on pod shutdown"

---

## Troubleshooting

### KVBM Not Initializing

If you see errors about KVBM initialization:

In [ ]:
%%bash
# Check KVBM worker logs
KVBM_POD=$(kubectl get pods -n $NAMESPACE -l nvidia.com/dynamo-namespace=vllm-kvbm-enabled,nvidia.com/dynamo-component-type=worker -o jsonpath='{.items[0].metadata.name}')

kubectl logs -n $NAMESPACE $KVBM_POD --tail=100

# Verify etcd and NATS are running from Lab 1
kubectl get pods -n $NAMESPACE | grep -E "(etcd|nats)"

# Check KVBM environment variables in pod
kubectl exec -n $NAMESPACE $KVBM_POD -- env | grep DYN_KVBM

### High Memory Usage

If system memory is exhausted:

In [ ]:
%%bash
# Edit deployment to reduce CPU cache size
# Change DYN_KVBM_CPU_CACHE_GB from "20" to "8" in kvbm_enabled.yaml
# Then reapply:
kubectl apply -f kvbm_enabled.yaml -n $NAMESPACE

# Check memory usage on worker pod
KVBM_POD=$(kubectl get pods -n $NAMESPACE -l nvidia.com/dynamo-namespace=vllm-kvbm-enabled,nvidia.com/dynamo-component-type=worker -o jsonpath='{.items[0].metadata.name}')
kubectl exec -n $NAMESPACE $KVBM_POD -- free -h

### Disk Offloading Issues

If disk offloading fails:

In [ ]:
%%bash
# Ensure directory exists and is writable on the node
# You may need cluster admin access for this
kubectl debug node/<node-name> -it --image=ubuntu -- bash
# Then inside the debug pod:
# mkdir -p /host/mnt/nvme/kvbm_cache
# chmod 755 /host/mnt/nvme/kvbm_cache

# Check available disk space from worker pod
KVBM_POD=$(kubectl get pods -n $NAMESPACE -l nvidia.com/dynamo-namespace=vllm-kvbm-enabled,nvidia.com/dynamo-component-type=worker -o jsonpath='{.items[0].metadata.name}')
kubectl exec -n $NAMESPACE $KVBM_POD -- df -h /mnt/nvme/kvbm_cache

# Check pod events for volume mount issues
kubectl describe pod -n $NAMESPACE $KVBM_POD | grep -A 10 Events

### Slow Benchmark Performance

If benchmarks are unexpectedly slow:

In [ ]:
%%bash
# Check worker pod resource usage
KVBM_POD=$(kubectl get pods -n $NAMESPACE -l nvidia.com/dynamo-namespace=vllm-kvbm-enabled,nvidia.com/dynamo-component-type=worker -o jsonpath='{.items[0].metadata.name}')

# Check GPU utilization
kubectl exec -n $NAMESPACE $KVBM_POD -- nvidia-smi

# Check CPU throttling
kubectl exec -n $NAMESPACE $KVBM_POD -- lscpu | grep MHz

# For faster testing, reduce prefix size:
# Use --prefix-prompt-length 10000 instead of 20000 in aiperf command

---

## Summary

### What You Learned
- ✅ How KVBM extends KV cache capacity beyond GPU memory
- ✅ Deploying baseline and KVBM-enabled serving configurations
- ✅ Benchmarking prefix cache reuse with realistic workloads
- ✅ Analyzing performance differences between offloading strategies
- ✅ Configuring multi-tier memory management (GPU + CPU + Disk)

### Key Takeaways
- **KVBM** eliminates redundant prefix recomputation by preserving blocks in CPU/disk
- **Prefix reuse** workloads can see 30-75% TTFT improvement with KVBM
- **Multi-tier offloading** enables handling far more concurrent requests
- **CPU cache** provides fast offload target; disk provides massive capacity
- **Best for workloads** with long, reusable prefixes (system prompts, documents, conversations)

### Performance Summary

| Aspect | Baseline (NIXL) | KVBM (20GB CPU) | Improvement |
|--------|----------------|-----------------|-------------|
| TTFT Average | 3,200 ms | 1,800 ms | **44% faster** |
| TTFT p99 | 12,500 ms | 3,200 ms | **74% faster** |
| Throughput | 6.2 req/sec | 8.8 req/sec | **42% higher** |
| Consistency | High variance | Low variance | **Much more stable** |

### Next Steps
- Combine KVBM with KV-aware routing from the previous lab for maximum efficiency
- Profile your production workload to determine optimal tier sizes
- Set up monitoring with Prometheus and Grafana dashboards
- Experiment with different models and prefix patterns

---

## Additional Resources

- 📖 [KVBM Architecture Documentation](../../dynamo/docs/architecture/kvbm_architecture.md)
- 🔧 [KVBM Configuration Guide](../../dynamo/docs/guides/run_kvbm_in_vllm.md)
- 📊 [NIXL Memory Transfer Layer](https://github.com/ai-dynamo/nixl)
- 🎯 [LMBenchmark Testing Framework](https://github.com/LMCache/LMBenchmark)

---

**Congratulations! 🎉** You've successfully compared baseline and KVBM-enabled serving!
